In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
from scipy.stats import norm
import scipy.integrate as integrate

import gym
from gym import spaces

import random
import itertools as it
from joblib import Parallel, delayed
from toolz import memoize
from contracts import contract
from collections import namedtuple, defaultdict, deque, Counter

import warnings
warnings.filterwarnings("ignore", 
                        message="The objective has been evaluated at this point before.")

from agents import Agent
from oldmouselab import OldMouselabEnv
from policies import FixedPlanPolicy, LiederPolicy
from evaluation import *
from distributions import cmax, smax, sample, expectation, Normal, PointMass, SampleDist, Normal, Categorical

In [4]:
def hd_dist(attributes):
    dist = [1,]*attributes
    dist[0] = np.random.randint(85,97)
    for i in range(1,attributes-1):
        dist[i] += np.random.randint(0,100-np.sum(dist))
    dist[-1] += 100-np.sum(dist)
    dist = np.around(np.array(dist)/100,decimals=2)
    np.random.shuffle(dist)
    return dist

def ld_dist(attributes):
    dist = [np.random.randint(10,40) for _ in range(attributes)]
    dist = np.around(np.array(dist)/sum(dist),decimals=2)
    np.random.shuffle(dist)
    return dist

In [13]:
gambles = 7
attributes = 4
high_stakes = Normal((9.99+0.01)/2, 0.3*(9.99-0.01))
low_stakes = Normal((0.25+0.01)/2, 0.3*(0.25-0.01))
reward = high_stakes
cost=.03

#set to 20 for sanity check
n_train = 20
n_test = 20

test_envs_hd =  [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_test)]
test_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_test)]
test_envs_all = test_envs_hd+test_envs_ld 

term_action = train_envs[0].term_action

In [14]:
agent = Agent()
def run_env(policy, env):
    agent.register(env)
    agent.register(policy)
    tr = agent.run_episode()
#     print(tr)
    return {'util': tr['return'], 'actions': tr['actions'],
            'observations': len(tr['actions']) - 1, 'ground_truth': env.ground_truth}

# High stakes, high dispersion

In [11]:
bo_pol_theta = np.load('data/high_stakes_3cents.npy')
bo_pol = LiederPolicy(list(bo_pol_theta))

In [ ]:
operations = np.array([])
nr_clicks = np.array([])
probabilities = np.array([])
for i in range(len(test_envs_hd)):
    train_envs_hd[i].reset()
    trace = run_env(bo_pol, test_envs_hd[i]) 
    operations = np.append(operations, trace['actions']) 
    nr_clicks = np.append(nr_clicks,[len(trace['actions'])-1])
    probabilities = np.append(probabilities,test_envs_hd[i].dist)

In [28]:
avg_nr_clicks=np.mean(nr_clicks)
min_nr_clicks=np.min(nr_clicks)
max_nr_clicks=np.max(nr_clicks)

print('The BO policy made between {} and {} clicks. The average number of clicks was {:3.2}'.format(min_nr_clicks,max_nr_clicks,avg_nr_clicks))

The BO policy made between 3.0 and 7.0 clicks. The average number of clicks was 6.3


In [10]:
train_envs[30].dist

array([ 0.09,  0.33,  0.32,  0.26])

In [25]:
train_envs[30].dist

array([ 0.13,  0.27,  0.3 ,  0.3 ])

# High stakes, low dispersion

In [ ]:
bo_pol_theta = np.load('data/high_stakes_3cents.npy')
bo_pol = LiederPolicy(list(bo_pol_theta))

# Low stakes, high dispersion

In [ ]:
bo_pol_theta = np.load('data/low_stakes_3cents.npy')
bo_pol = LiederPolicy(list(bo_pol_theta))

# Low stakes, low dispersion

In [12]:
bo_pol_theta = np.load('data/low_stakes_3cents.npy')
bo_pol = LiederPolicy(list(bo_pol_theta))

FileNotFoundError: [Errno 2] No such file or directory: 'data/low_stakes_3cents.npy'